# Prepare Data

### Load Data

In [1]:
import numpy as np
import pandas as pd

from gbd_tool.gbd_api import GBD

# Handle custom error codes
def convert_value(x: str) -> float:
    if x is None or x == "empty" or x == "failed" or x == "memout" or x == "unverified":
        return np.nan
    elif x == "timeout":
        return np.inf
    else:
        try:
            return float(x)
        except ValueError as e:
            print(x)
            raise e

# Meta-data dataframe
with GBD(["../al-for-sat-solver-benchmarking-data/gbd-data/meta.db"]) as gbd:
    features = gbd.get_features()
    features.remove("hash")
    meta_df = pd.DataFrame.from_records(
        gbd.query_search("", resolve=features),
        columns=gbd.get_features())

# Dataframe with all base features
with GBD([
    "../al-for-sat-solver-benchmarking-data/gbd-data/meta.db",
    "../al-for-sat-solver-benchmarking-data/gbd-data/base.db"
]) as gbd:
    features = gbd.get_features()
    for meta_feat in gbd.get_features(dbname="meta_db"):
        features.remove(meta_feat)
    features_with_hash = features.copy()
    features_with_hash.insert(0, "hash")
    base_features_df = pd.DataFrame.from_records(
        gbd.query_search("", resolve=features),
        columns=features_with_hash)
    
# Coerce into correct datatype and set index
for col_name in base_features_df.columns:
    if col_name == "hash":
        base_features_df[col_name] = base_features_df[col_name].astype("string")
    else:
        base_features_df[col_name] = base_features_df[col_name].map(convert_value)   
base_features_df.set_index("hash", inplace=True, drop=True)
meta_df.set_index("hash", inplace=True, drop=True)

# Only use instances with all feature values present
nan_feature_rows = np.any(np.isnan(base_features_df.replace([np.inf, -np.inf], np.nan)), axis=1)
base_features_df = base_features_df[~nan_feature_rows]
meta_df = meta_df.loc[base_features_df.index, :]

# Drop zero variance features
base_features_df.drop(base_features_df.columns[base_features_df.std() == 0], axis=1, inplace=True)

meta_df.shape, base_features_df.shape

((29353, 9), (29353, 46))

### Save data

In [2]:
np.all(base_features_df.index == meta_df.index), base_features_df.shape, meta_df.shape

(True, (29353, 46), (29353, 9))

In [3]:
import pickle

with open("../al-for-sat-solver-benchmarking-data/pickled-data/meta_df.pkl", "wb") as file:
    pickle.dump(meta_df.copy(), file)
with open("../al-for-sat-solver-benchmarking-data/pickled-data/base_features_df.pkl", "wb") as file:
    pickle.dump(base_features_df.copy(), file)

## Anniversay Data

In [4]:
anni_df = pd.read_csv("../al-for-sat-solver-benchmarking-data/gbd-data/anni-seq.csv", index_col="hash")
anni_df.drop(["benchmark", "verified-result", "claimed-result"], axis=1, inplace=True)
anni_df = anni_df[anni_df.index.isin(meta_df.index)].copy()
anni_df = anni_df.replace(10000.0, np.inf).copy()
anni_df.shape

(5301, 28)

In [5]:
from sklearn.model_selection import train_test_split

# Assign family label other to small families
anni_meta_df = meta_df.loc[anni_df.index, "family"]
all_families = anni_meta_df.unique()
small_families_mask = anni_meta_df.value_counts(sort=False) < 10
small_families = all_families[small_families_mask]
stratifcation_values = np.where(anni_meta_df.isin(small_families), "other", anni_meta_df)

anni_train_df = train_test_split(
    anni_df, test_size=0.1,
    stratify=stratifcation_values,
    shuffle=True, random_state=42,
)[1]

In [6]:
# Families in complete anniversary track dataset
meta_df.loc[anni_df.index, "family"].value_counts()

hardware-verification           344
planning                        333
cryptography                    311
bitvector                       214
quasigroup-completion           210
                               ... 
karatsuba-multiplication          2
maximum-constraint-partition      2
circuit-minimization              2
glassy-gen                        1
ramsey                            1
Name: family, Length: 133, dtype: int64

In [7]:
# Families in smaller hyper-parameter optimization dataset
meta_df.loc[anni_train_df.index, "family"].value_counts()

hardware-verification    35
planning                 33
cryptography             31
bitvector                22
quasigroup-completion    21
                         ..
sum-of-3-cubes            1
core-based-generator      1
rubikcube                 1
edit-distance             1
mutilated-chessboard      1
Name: family, Length: 112, dtype: int64

In [8]:
import pickle

with open("../al-for-sat-solver-benchmarking-data/pickled-data/anni_train_df.pkl", "wb") as file:
    pickle.dump(anni_train_df.copy(), file)

## Final Anniversary Data

In [9]:
anni_df = pd.read_csv("../al-for-sat-solver-benchmarking-data/gbd-data/anni-seq.csv", index_col="hash")
anni_df.drop(["benchmark", "verified-result", "claimed-result"], axis=1, inplace=True)
anni_df = anni_df[anni_df.index.isin(base_features_df.index)].copy()
anni_df = anni_df.replace(10000.0, np.inf).copy()
anni_df.shape

(5301, 28)

In [10]:
pd.DataFrame.from_records(list(sorted(zip(
    anni_df.columns,
    np.mean(anni_df.replace(np.inf, 10000), axis=0)
), key=lambda x: x[1])), columns=["Algorithm", "PAR-2 score"]).set_index("Algorithm", drop=True)

,PAR-2 score
Algorithm,
Kissat_MAB_ESA,2808.131702
kissat-sc2022-bulky,2812.931160
ekissat-mab-gb-db,2835.247538
Kissat_MAB_UCB,2835.594988
kissat_inc,2836.915263
ekissat-mab-db-v1,2845.189361
Kissat_MAB_MOSS,2846.732671
Kissat_MAB-HyWalk,2857.666290
kissat-sc2022-light,2869.454962


In [11]:
with open("../al-for-sat-solver-benchmarking-data/pickled-data/anni_final_df.pkl", "wb") as file:
    pickle.dump(anni_df.copy(), file)

In [12]:
import numpy as np
import pandas as pd
import pickle

anni_full_df = pd.read_csv(
    "../al-for-sat-solver-benchmarking-data/gbd-data/anni-seq.csv", index_col="hash")
anni_full_df.drop(["benchmark", "verified-result", "claimed-result"], axis=1, inplace=True)
anni_full_df = anni_full_df.replace(10000.0, np.inf).copy()

with open("../al-for-sat-solver-benchmarking-data/pickled-data/anni_full_df.pkl", "wb") as file:
    pickle.dump(anni_full_df.copy(), file)

anni_full_df.shape

(5355, 28)